In [3]:
"""
This module implements the TSP Problem

State: Current path through the cities, starting
       and ending at the same city
Action: Swap two cities within the path (not the
        start/end city) to create a new path through
        the cities

@author: Sean Brouwer
@version 2/22/19
"""
from tools.aima.search import Problem, hill_climbing,\
    simulated_annealing, exp_schedule, genetic_search
import random
import time


class TSP(Problem):
    """
    State: Path covering all cities
    Move: Swap two cities in path
    """

    def __init__(self, start_path, distance_dict):
        self.initial = start_path
        self.distance_dict = distance_dict
        
    def actions(self, current_path):
        swappable_cities = current_path[1:-2]
        action_list = []
        for city_1 in swappable_cities:
            for city_2 in swappable_cities:
                if (city_2, city_1) not in action_list and\
                        city_1 != city_2:
                    action = (city_1, city_2)
                    action_list.append(action)
        return action_list
        
    def result(self, current_path, action):
        index_1 = current_path.index(action[0])
        index_2 = current_path.index(action[1])
        city_1 = current_path[index_1]
        city_2 = current_path[index_2]
        new_path = current_path
        new_path[index_1] = city_2
        new_path[index_2] = city_1
        return new_path
    
    def value(self, current_path):
        path_cost = 0
        for i in range(len(current_path) - 1):
            if (current_path[i], current_path[i+1])\
                    in self.distance_dict:
                path_cost += self.distance_dict[
                    current_path[i], current_path[i+1]]
            else:
                path_cost += self.distance_dict[
                    current_path[i+1], current_path[i]]
        return path_cost


if __name__ == '__main__':
    # Formulate a TSP
    # Setup initial city order
    numCities = 20
    startPath = []
    for i in range(numCities):
        startPath.append(i)
    random.shuffle(startPath)
    startPath.append(startPath[0])
    print("Starting Path: " + str(startPath))
    
    # Setup distance dictionary
    distanceDict = {}
    for city1 in startPath:
        for city2 in startPath:
            if (city2, city1) not in distanceDict and\
                    city1 != city2:
                dictEntry = (city1, city2)
                distanceDict[dictEntry] =\
                    random.randrange(1, 5, 1)
    # print("Distance Dictionary:")
    # print(str(distanceDict))
    
    p = TSP(startPath, distanceDict)

    # Solve the problem using hill-climbing.
    print("\nHill Climbing:")
    hcStartTime = time.time()
    hill_solution = hill_climbing(p)
    hcEndTime = time.time()
    print("Best Path: " + str(hill_solution))
    print("Best Path cost: " + str(p.value(hill_solution)))
    print("Execution time: " + str(hcEndTime-hcStartTime))

    # Solve the problem using simulated annealing.
    print("\nSimulated Annealing:")
    annealingStartTime = time.time()
    annealing_solution = simulated_annealing(
        p,
        exp_schedule(k=20, lam=0.005, limit=1000)
    )
    annealingEndTime = time.time()
    print("Best Path: " + str(annealing_solution))
    print("Best Path cost: " +
          str(p.value(annealing_solution)))
    print("Execution time: " +
          str(annealingEndTime-annealingStartTime))


Starting Path: [16, 10, 4, 0, 13, 1, 17, 5, 19, 18, 7, 6, 9, 2, 11, 3, 12, 14, 8, 15, 16]

Hill Climbing:
Best Path: [16, 8, 14, 12, 3, 11, 2, 9, 6, 7, 18, 19, 5, 17, 1, 13, 0, 4, 10, 15, 16]
Best Path cost: 34
Execution time: 0.0030050277709960938

Simulated Annealing:


Best Path: [16, 8, 14, 12, 3, 11, 2, 9, 6, 7, 18, 19, 5, 17, 1, 13, 0, 4, 10, 15, 16]
Best Path cost: 34
Execution time: 1.6987624168395996


In [14]:
"""
This module implements a course-scheduling problem in
a constraint satisfaction framework

Domains: Courses (CS104, CS108, CS112, CS212, CS214, CS344)
Variables: Faculty (Adams, VanderLinden, Plantinga, Norman)
           Time Slots (900, 1030, 1130, 1230)
           Classrooms (NH253, SB382)

@author: Sean Brouwer
@version 2/22/19
"""

from tools.aima.csp import CSP, AC3, backtracking_search,\
    min_conflicts, parse_neighbors


def course_scheduling():
    """Return an instance of the Scheduling Problem."""
    courses = 'CS104 CS108 CS112 CS212 CS214 CS344'.split()
    faculty = 'Adams1 Adams2 VanderLinden Plantinga Norman1 Norman2'.split()
    # time_slots = '900 1030 1130 1230'.split()
    # classrooms = 'NH253 SB382'.split()
    variables = faculty + courses
    domains = {}
    for var in variables:
        domains[var] = ['NH253 9:00', 'SB382 9:00',
                        'NH253 10:30', 'SB382 10:30',
                        'NH253 11:30', 'SB382 11:30',
                        'NH253 12:30', 'SB382 12:30']
    neighbors = parse_neighbors("""Norman1: CS104;
                Norman2: CS108; Adams1: CS112; Plantinga: CS212;
                Adams2: CS214; VanderLinden: CS344""", variables)
    for type in [courses, faculty]:
        for A in type:
            for B in type:
                if A != B:
                    if B not in neighbors[A]:
                        neighbors[A].append(B)
                    if A not in neighbors[B]:
                        neighbors[B].append(A)

    def schedule_constraint(A, a, B, b, recurse=0):
        same = (a == b)
        next_to = (a[6:8] == b[6:8])
        if A == 'Norman1' and B == 'CS104':
            return same
        if A == 'Norman2' and B == 'CS108':
            return same
        if A == 'Adams1' and B == 'CS112':
            return same
        if A == 'Plantinga' and B == 'CS212':
            return same
        if A == 'Adams2' and B == 'CS214':
            return same
        if A == 'VanderLinden' and B == 'CS344':
            return same
        if A == 'Norman1' and B == 'Norman2':
            return not next_to
        if A == 'Norman2' and B == 'Norman1':
            return not next_to
        if A == 'Adams1' and B == 'Adams2':
            return not next_to
        if A == 'Adams2' and B == 'Adams1':
            return not next_to
        if recurse == 0:
            return schedule_constraint(B, b, A, a, 1)
        if ((A in courses and B in courses) or
                (A in faculty and B in faculty)):
            return not same
        raise Exception('error')
    return CSP(variables, domains, neighbors, schedule_constraint)


def solve_schedule(algorithm=min_conflicts, **args):
    sch = course_scheduling()
    ans = algorithm(sch, **args)
    if type(ans) == bool:
        print("Unable to solve")
        return
    for loc in ['NH253 9:00', 'NH253 10:30', 'NH253 11:30',
                'NH253 12:30', 'SB382 9:00', 'SB382 10:30',
                'SB382 11:30', 'SB382 12:30']:
        print(loc, end=' ')
        for (var, val) in ans.items():
            if val == loc:
                print(var, end=' ')
        print()
    
    
if __name__ == '__main__':
    print("Min_conflicts:")
    solve_schedule()
    print("\nAC3:")
    solve_schedule(AC3)
    print("\nBacktracking_search:")
    solve_schedule(backtracking_search)


Min_conflicts:
NH253 9:00 
NH253 10:30 
NH253 11:30 VanderLinden CS344 
NH253 12:30 Plantinga CS212 
SB382 9:00 Adams2 CS214 
SB382 10:30 Norman1 CS104 
SB382 11:30 Norman2 CS108 
SB382 12:30 Adams1 CS112 

AC3:
Unable to solve

Backtracking_search:
NH253 9:00 Adams1 CS112 
NH253 10:30 Adams2 CS214 
NH253 11:30 Norman1 CS104 
NH253 12:30 Norman2 CS108 
SB382 9:00 VanderLinden CS344 
SB382 10:30 Plantinga CS212 
SB382 11:30 
SB382 12:30 
